In [1]:
import nltk
import numpy as np
import operator as op
import pandas as pd
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import io
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_train = '/content/drive/My Drive/Analyse_des_sentiment/data/train.xml'
file_dev   = '/content/drive/My Drive/Analyse_des_sentiment/data/dev.xml'
file_test  = '/content/drive/My Drive/Analyse_des_sentiment/data/test.xml'

In [4]:
def read_xml_to_dataframe(file_path):
    # Analyse du fichier XML
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Création d'une liste vide pour stocker les données
    data = []

    # Parcours des éléments de commentaire
    for comment in root.findall('comment'):
        movie = comment.find('movie').text
        review_id = comment.find('review_id').text
        name = comment.find('name').text
        user_id = comment.find('user_id').text
        note = comment.find('note').text.replace(',', '.')
        commentaire = comment.find('commentaire').text

        # Ajout des données dans la liste sous forme de dictionnaire
        data.append({
            'movie': movie,
            'review_id': review_id,
            'name': name,
            'user_id': user_id,
            'note': note,
            'commentaire': commentaire
        })

    # Création d'un DataFrame
    df = pd.DataFrame(data)

    return df

In [5]:
#fonction pour le fichier de test
def read_xml_to_dataframeTest(file_path):
    # Analyse du fichier XML
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Création d'une liste vide pour stocker les données
    data = []

    # Parcours des éléments de commentaire
    for comment in root.findall('comment'):
        # Initialisation d'un dictionnaire pour chaque commentaire
        comment_data = {}

        # Extraction de chaque élément si disponible et ajout au dictionnaire
        movie = comment.find('movie')
        if movie is not None:
            comment_data['movie'] = movie.text

        review_id = comment.find('review_id')
        if review_id is not None:
            comment_data['review_id'] = review_id.text

        name = comment.find('name')
        if name is not None:
            comment_data['name'] = name.text

        user_id = comment.find('user_id')
        if user_id is not None:
            comment_data['user_id'] = user_id.text

        # Note is not present in the given data, but if it's sometimes included,
        # you can check for it like this:
        note = comment.find('note')
        if note is not None:
            comment_data['note'] = note.text.replace(',', '.')

        commentaire = comment.find('commentaire')
        if commentaire is not None:
            comment_data['commentaire'] = commentaire.text

        # Ajout des données dans la liste sous forme de dictionnaire
        data.append(comment_data)

    # Création d'un DataFrame
    df = pd.DataFrame(data)

    return df

In [6]:
#création DataFrame
data_dev   = read_xml_to_dataframe(file_dev)
data_train = read_xml_to_dataframe(file_train)
data_test  = read_xml_to_dataframeTest(file_test)

In [7]:
data_train['commentaire'] = data_train['commentaire'].fillna('')
data_dev['commentaire'] = data_dev['commentaire'].fillna('')
data_test['commentaire'] = data_test['commentaire'].fillna('')

In [8]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
!python -m spacy download fr_core_news_sm

2024-01-08 13:15:00.344899: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 13:15:00.344973: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 13:15:00.346449: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 13:15:00.355018: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-08 13:15:01.603633: W tensorflow/compiler/tf2

In [10]:
import spacy
import re
# Load the French language model
nlp = spacy.load('fr_core_news_sm')

In [11]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Tokenization and lemmatization using spaCy, and removal of stop words
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.is_alpha and token.text not in spacy.lang.fr.stop_words.STOP_WORDS]

    return tokens

In [13]:
preprocess_text("hy j'adore ..; si s'il y a plein  pas le film")

['hy', 'adorer', 'plein', 'film']

In [14]:
data_train['commentaire_w_processed'] = data_train['commentaire'].apply(preprocess_text)
data_train.to_csv('/content/drive/My Drive/processed_data/data_train_ww_processed.csv', index=False)

In [15]:
data_dev['commentaire_w_processed'] = data_dev['commentaire'].apply(preprocess_text)
data_train.to_csv('/content/drive/My Drive/processed_data/data_dev_ww_processed.csv', index=False)

data_test['commentaire_w_processed'] = data_test['commentaire'].apply(preprocess_text)
data_train.to_csv('/content/drive/My Drive/processed_data/data_test_ww_processed.csv', index=False)


In [17]:
data_train

,movie,review_id,name,user_id,note,commentaire,commentaire_w_processed
0,229864,review_58145147,AM11,Z20051105131449940367618,4.0,"""Ce n'est pas ce qui se trouve en face de vous...","[trouver, face, tenir, côté, phrase, tagline, ..."
1,229864,review_59303833,this is my movies,Z20051220181957240298347,3.5,"(...) En soi, je n'ai rien contre ce genre de ...","[rien, contre, genre, récit, hyper, codifier, ..."
2,229864,review_61916672,Shephard69,Z20121226180301760912584,4.5,"Pour sa troisième réalisation après ""Tron, l'h...","[réalisation, tron, héritage, oblivion, attend..."
3,229864,review_1000190786,Tayop,Z20121022182542230468657,3.0,Joseph Kosinski offre de nouvelles images ébou...,"[joseph, kosinski, offrir, nouveau, image, ébo..."
4,229864,review_59207755,ned123,Z20061211223247607682175,3.5,J'ai vu un film... qui traite d'un drame humai...,"[film, traire, drame, humain, haute, intensité..."
...,...,...,...,...,...,...,...
665957,245551,review_55820729,Wally L,Z20121109004006553333277,4.0,super realisation et quelques bon acteurs. ce...,"[super, realisation, bon, acteur, ville, richm..."
665958,245551,review_1001840194,r@ch,Z20100726110927837577369,3.0,Film visuellement très beau et mis en musique ...,"[film, visuellemer, beau, mettre, musique, rap..."
665959,245551,review_56242470,Kaiser D.,Z20170721223303710182536,3.5,parti d'une histoire qui parait pour nous bana...,"[partir, histoire, parer, banal, bon, film, je..."
665960,245551,review_58174196,Florent A.,Z20161204004443590736237,3.0,"Je suis mitigé sur ce film.d un coté,j ai plut...","[mitiger, coté, j, j, être, mal, l, aise, long..."


In [18]:
from gensim.models import Word2Vec

# Train Word2Vec model on your preprocessed training data
model = Word2Vec(sentences=data_train['commentaire_w_processed'], vector_size=100, window=5, min_count=1, sg=0)

# Save the Word2Vec model to a file
model.save("word2vec.model")


In [19]:
# Load your Word2Vec model
model = Word2Vec.load("word2vec.model")

In [20]:
# Step 1: Convert tokens to Word2Vec embeddings
def tokens_to_embeddings(tokens, model):
    embeddings = []
    for token in tokens:
        if token in model.wv:
            embeddings.append(model.wv[token])
    return embeddings

In [21]:
# Step 1: Convert tokens to Word2Vec embeddings
data_train['embeddings'] = data_train['commentaire_w_processed'].apply(lambda x: tokens_to_embeddings(x, model))
data_dev['embeddings'] = data_dev['commentaire_w_processed'].apply(lambda x: tokens_to_embeddings(x, model))
data_test['embeddings'] = data_test['commentaire_w_processed'].apply(lambda x: tokens_to_embeddings(x, model))

In [22]:
# Step 2: Calculate average embeddings for each comment
data_train['avg_embeddings'] = data_train['embeddings'].apply(lambda x: np.mean(x, axis=0) if x else np.zeros(model.vector_size))
data_dev['avg_embeddings'] = data_dev['embeddings'].apply(lambda x: np.mean(x, axis=0) if x else np.zeros(model.vector_size))
data_test['avg_embeddings'] = data_test['embeddings'].apply(lambda x: np.mean(x, axis=0) if x else np.zeros(model.vector_size))

In [28]:
# Step 3: Convert data to SVM format
train_svm_data = data_train[['avg_embeddings', 'note']]
dev_svm_data = data_dev[['avg_embeddings', 'note']]
test_svm_data = data_test['avg_embeddings']


In [37]:
test_svm_data

0        [0.12871014, -0.08519242, -0.17973067, -0.5283...
1        [0.28309113, -0.24553326, -0.063483626, -0.990...
2        [0.0817979, -0.41899243, -0.17068678, -0.73497...
3        [0.23239677, -0.5388362, 0.320079, -0.33318806...
4        [-0.12405506, -0.35570562, 0.28846765, -0.6477...
                               ...                        
85842    [0.17296925, 0.8837883, -0.054136194, -0.68306...
85843    [-0.016820129, -0.28371003, -0.4178806, -1.005...
85844    [0.4722395, -0.017239062, -0.26149806, -1.2264...
85845    [-0.21894524, -0.13637803, -0.12301482, -1.176...
85846    [0.7852612, -0.0577177, -0.08140442, -0.377744...
Name: avg_embeddings, Length: 85847, dtype: object

In [44]:
# Define the default label for test data


def convertir_svm_from_dict(data_dict, etiquettes=None, default_label=0):
    svm = ""
    print("convertir_svm")
    for i, features in enumerate(data_dict):
        # Convert label to integer by multiplying by 2
        label = int(float(etiquettes[i]) * 2) if etiquettes is not None and i in etiquettes else default_label
        svm += str(label) + " "

        # Sort the features by index in ascending order
        sorted_features = sorted(enumerate(features, start=1), key=lambda x: x[0])

        # Convert sorted features to SVM format
        for j, value in sorted_features:
            svm += f"{j}:{value} "
        svm += "\n"
    return svm




In [38]:
# Use your existing functions to convert to SVM format
train_svm = convertir_svm_from_dict(train_svm_data['avg_embeddings'].to_dict(), train_svm_data['note'])

convertir_svm


In [45]:
test_svm = convertir_svm_from_dict(test_svm_data.to_list(),default_label=0)

convertir_svm


In [48]:
def file(nom,contenu):

    # Ouvrez le fichier en mode écriture
    with io.open(nom, "w", encoding="utf-8") as f:
        # Écrivez vos lignes de string dans le fichier
        f.writelines(contenu)


In [49]:
file("/content/drive/My Drive/processed_data/train_hope.svm", train_svm)

In [52]:
test_svm

In [53]:
file("/content/drive/My Drive/processed_data/test_hope.svm", test_svm)